In [1]:
# md5 : 속도보다 보안 but 보안도 이제는 떨어짐
import hashlib
def md5(file1):
    md5h = hashlib.md5()
    with open(file1, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            md5h.update(chunk)
    return md5h.hexdigest()

In [2]:
import os
from tqdm.notebook import tqdm
import pandas as pd
import file_function as ff
import re
import binascii
import pickle

In [2]:
def crc32_checksum(filename):
    buf = open(filename,'rb').read()
    buf = (binascii.crc32(buf) & 0xFFFFFFFF)
    return "%08X" % buf

In [3]:
def file_info(path:str) -> dict:
    """
    하위경로포함 모든 파일에 대해 size,cre32로 중복 검사 후 
    모든 파일 정보 및 중복파일 목록을 excel, pickle로 바탕화면에 저장
    바로 중복파일 관리할 수 있도록 중복목록인 dict_sc 반환
    """

    dict_sc = {}  #중복파일끼리 묶을 딕셔너리
    df = None
    data = []
    
    df_matching = pd.read_excel(
        r'D:\workspace\python_work\파일\매각사 이름매칭.xlsx')

    for root, __dirs__, files in tqdm(os.walk(path)):
        
        for f in files:
            fullname = os.path.join(root, f)
            #key
            size = str(os.path.getsize(fullname))
            #value
            mtime = str(os.path.getmtime(fullname))
            crc32 = str(crc32_checksum(fullname))
            sell = ""
            for __index__, row in df_matching.iterrows():
                if re.search(str(row[0]), str(root)):  
                    sell = row[1]
                    break            
            stem = os.path.splitext(f)[0]
            ext = os.path.splitext(f)[1]
            length = len(stem)
            
            temp = {"size":size, "crc32": crc32, "mtime":mtime, "sell":sell, "root":root, "stem":stem, "ext":ext, "length":length, "fullname":fullname}
            data.append(temp)
            
            # 고유한 size, crc를 키로 하는 딕셔너리 만들기
            sc = size + crc32
            if sc not in dict_sc:
                dict_sc[sc] = [temp]
            else:
                dict_sc[sc].append(temp)
            
            
    df = pd.DataFrame(data)
    
    # 파일 내보내기1
    df.to_excel(r"C:\Users\SL\Desktop\파일정보.xlsx")
    df.to_pickle(r"C:\Users\SL\Desktop\파일정보.pkl")

    # 파일 내보내기2 : 중복파일만 모은 거
    for key, value in dict_sc.items():
        if len(value) == 1 :
            del dict_sc[key]

    with open(r"C:\Users\SL\Desktop\파일정보_sc.pkl") as pk :
        pickle.dump(dict_sc, pk)
            
    return dict_sc

In [4]:
path = r'\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채권양도통지서'
dict_sc, dict_sm, dict_scm = file_info(path)

0it [00:00, ?it/s]

In [4]:
# 파일 관리하기 
df = pd.read_excel(r'\\192.168.10.83\0.전산\파일정보.xlsx') #일단 pkl 없으니
sc = df.duplicated(['size', 'crc32'], keep=False)
sc.name = "sc"
df = pd.concat([df, sc], axis=1)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df_sc = df[df["sc"]]
df_sc.to_pickle(r"D:\\파일정보_sc.pkl")

In [10]:
df2 = pd.read_pickle(r"D:\scan\파일정보_sc.pkl")
print(len(df2))
df2.head(1)

10509


,size,crc32,mtime,sell,root,stem,ext,length,sc
5,109306,1252DB17,1.378795e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,20417854 (유)가람관광여행사,.tif,19,True


In [11]:
import shutil

for index, row in df2.head(20).iterrows():
    fullname = row.root +"/"+row.stem + row.ext
    dst = r'D:\scan'+"/" + row.stem + row.ext
    shutil.copy2(fullname, dst)

In [52]:
import pandas as pd
dict = [{
 'apple': 3,
 'banana': 5,
 'mango': 7,
 'apricot': 1,
 'kiwi': 8,
 'orange': 3}, {'apple': 3,
 'banana': 5,
 'mango': 7,
 'apricot': 1,
 'kiwi': 8,
 'orange': 3}]
pd.DataFrame(dict)

,apple,banana,mango,apricot,kiwi,orange
0,3,5,7,1,8,3
1,3,5,7,1,8,3


In [16]:
import numpy as np
numOfRows = 50000 
def dict_list_comprehension():
    data = [{(a, np.random.randint(100)) for a in ['A', 'B', 'C', 'D', 'E']} for i in range(numOfRows)]
    df = pd.DataFrame.from_dict(data)
    return data, df

In [17]:
data, temp = dict_list_comprehension()

In [18]:
data

[{('A', 25), ('B', 78), ('C', 31), ('D', 83), ('E', 53)},
 {('A', 0), ('B', 69), ('C', 2), ('D', 37), ('E', 40)},
 {('A', 64), ('B', 50), ('C', 26), ('D', 26), ('E', 12)},
 {('A', 39), ('B', 58), ('C', 57), ('D', 26), ('E', 50)},
 {('A', 32), ('B', 77), ('C', 33), ('D', 3), ('E', 30)},
 {('A', 63), ('B', 22), ('C', 76), ('D', 37), ('E', 42)},
 {('A', 84), ('B', 12), ('C', 93), ('D', 15), ('E', 5)},
 {('A', 34), ('B', 80), ('C', 30), ('D', 32), ('E', 44)},
 {('A', 73), ('B', 35), ('C', 82), ('D', 81), ('E', 43)},
 {('A', 95), ('B', 87), ('C', 29), ('D', 77), ('E', 58)},
 {('A', 81), ('B', 69), ('C', 8), ('D', 76), ('E', 60)},
 {('A', 71), ('B', 58), ('C', 43), ('D', 28), ('E', 92)},
 {('A', 37), ('B', 56), ('C', 53), ('D', 12), ('E', 96)},
 {('A', 89), ('B', 31), ('C', 96), ('D', 21), ('E', 27)},
 {('A', 51), ('B', 16), ('C', 82), ('D', 10), ('E', 28)},
 {('A', 53), ('B', 11), ('C', 53), ('D', 59), ('E', 48)},
 {('A', 90), ('B', 32), ('C', 41), ('D', 84), ('E', 46)},
 {('A', 80), ('B', 

In [19]:
temp

,0,1,2,3,4
0,"(A, 25)","(E, 53)","(C, 31)","(D, 83)","(B, 78)"
1,"(A, 0)","(E, 40)","(C, 2)","(B, 69)","(D, 37)"
2,"(C, 26)","(E, 12)","(A, 64)","(D, 26)","(B, 50)"
3,"(A, 39)","(B, 58)","(E, 50)","(D, 26)","(C, 57)"
4,"(A, 32)","(B, 77)","(E, 30)","(D, 3)","(C, 33)"
...,...,...,...,...,...
49995,"(D, 65)","(A, 74)","(E, 98)","(B, 79)","(C, 56)"
49996,"(C, 15)","(B, 51)","(A, 79)","(D, 41)","(E, 67)"
49997,"(C, 48)","(E, 33)","(B, 14)","(D, 82)","(A, 81)"
49998,"(D, 24)","(B, 96)","(C, 73)","(A, 79)","(E, 67)"
